In [ ]:
%reload_ext autoreload
%autoreload 2

import functools

print = functools.partial(print, flush=True)

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import flexiznam as flz
from cottage_analysis.preprocessing import synchronisation
from cottage_analysis.analysis import spheres

In [ ]:
project = "hey2_3d-vision_foodres_20220101"
session_name = "PZAH6.4b_S20220419"
flexilims_session = flz.get_flexilims_session(project)
exp_session = flz.get_entity(
    datatype="session", name=session_name, flexilims_session=flexilims_session
)
recording_type = "two_photon"
recordings = flz.get_entities(
    datatype="recording",
    origin_id=exp_session["id"],
    query_key="recording_type",
    query_value=recording_type,
    flexilims_session=flexilims_session,
)
recording = flz.get_entity(
    datatype="recording",
    name=recordings.iloc[0].name,
    flexilims_session=flexilims_session,
)

vs_df = synchronisation.generate_vs_df(
    recording=recording,
    photodiode_protocol=2,
    flexilims_session=None,
    project="hey2_3d-vision_foodres_20220101",
)

imaging_df = synchronisation.generate_imaging_df(
    vs_df=vs_df,
    recording=recording,
    flexilims_session=flexilims_session,
    filter_datasets={"anatomical_only": 3},
    return_volumes=True,
)

imaging_df = spheres.format_imaging_df(recording=recording, imaging_df=imaging_df)

trials_df = spheres.generate_trials_df(recording=recording, imaging_df=imaging_df)

trials_df = spheres.search_param_log_trials(
    recording=recording, trials_df=trials_df, flexilims_session=flexilims_session
)

harp_ds = flz.get_datasets(
    flexilims_session=flexilims_session,
    origin_name=recording.name,
    dataset_type="harp",
    allow_multiple=False,
    return_dataseries=False,
)
paramlog_path = harp_ds.path_full / harp_ds.csv_files["NewParams"]
param_log = pd.read_csv(paramlog_path)

frames = spheres.regenerate_frames(
    frame_times=imaging_df.imaging_harptime,
    trials_df=trials_df,
    vs_df=vs_df,
    param_logger=param_log,
    time_column="HarpTime",
    resolution=5,
    sphere_size=10,
    azimuth_limits=(-120, 120),
    elevation_limits=(-40, 40),
    verbose=True,
    output_datatype="int16",
    output=None,
    flip_x=True,
)

In [ ]:
from cottage_analysis.analysis import find_depth_neurons

neurons_df = find_depth_neurons.find_depth_neurons(trials_df)

In [ ]:
stimulus_frames = np.sum(frames, axis=(1, 2)) > 0
dffs = np.concatenate(imaging_df.spks)
dffs = dffs - np.mean(dffs[stimulus_frames, :], axis=0)
stas = dffs.T @ np.roll(np.reshape(frames, (frames.shape[0], -1)), 1, axis=0)
stas = np.reshape(stas, (stas.shape[0], *frames.shape[1:]))
sum_frames = np.sum(frames, axis=0)
roi = 179
sta = stas[roi, :, :] / sum_frames
vmax = np.quantile(sta[np.isfinite(sta)], 0.999)
plt.imshow(sta, origin="lower", vmin=-vmax, vmax=vmax, cmap="bwr")

In [ ]:
reg_xy = [20, 40, 80, 160, 320]
reg_depth = [20, 40, 80, 160, 320]
r2s = np.zeros((len(reg_xy), len(reg_depth), imaging_df.iloc[0].dffs.shape[1], 2))
for i, reg_xy_ in enumerate(reg_xy):
    for j, reg_depth_ in enumerate(reg_depth):
        print(f"reg_xy: {reg_xy_}, reg_depth: {reg_depth_}")
        coef, r2 = spheres.fit_3d_rfs(
            imaging_df, frames[:, :, 24:], reg_xy=reg_xy_, reg_depth=reg_depth_
        )
        r2s[i, j, :, :] = r2

In [ ]:
plt.imshow(np.mean(r2s[:, :, :, 1] > 0.01, axis=2), origin="lower")
plt.xticks(range(len(reg_depth)), reg_depth)
plt.yticks(range(len(reg_xy)), reg_xy)
plt.xlabel("reg_depth")
plt.ylabel("reg_xy")
plt.colorbar()
plt.title("Fraction of neurons with R2>0.01")

In [ ]:
coef, r2 = spheres.fit_3d_rfs(
    imaging_df, frames[:, :, 24:], reg_xy=40, reg_depth=80, shift_stim=2
)
coef_ipsi, r2_ipsi = spheres.fit_3d_rfs(
    imaging_df, frames[:, :, :24], reg_xy=40, reg_depth=80, shift_stim=2
)

In [ ]:
np.mean(
    r2[neurons_df.depth_neuron_anova_p < 0.001, 1]
    > np.quantile(r2_ipsi[neurons_df.depth_neuron_anova_p < 0.001, 1], 0.95)
)

In [ ]:
depths = np.unique(imaging_df.depth)
depths = depths[~np.isnan(depths)]
depths = depths[depths > 0]
depths = np.sort(depths)
coef_mean = np.mean(np.stack(coef, axis=2), axis=2)

roi = 179
coef_fit, params = spheres.fit_3d_rfs_parametric(
    coef_mean[:, roi][:, np.newaxis],
    24,
    frames.shape[1],
    depths.shape[0],
    model="gabor",
)

c = np.reshape(coef_mean[:-1, roi], (depths.shape[0], frames.shape[1], 24))
c_fit = np.reshape(coef_fit[:-1, 0], (depths.shape[0], frames.shape[1], 24))

plt.figure(figsize=(5, 4))
vmax = np.max(np.abs(c))
for idepth, depth in enumerate(depths):
    plt.subplot(len(depths), 2, (idepth * 2) + 1)
    plt.imshow(c[idepth, :, :], origin="lower", vmin=-vmax, vmax=vmax, cmap="bwr")
    plt.axis("off")
    plt.subplot(len(depths), 2, (idepth * 2) + 2)
    plt.imshow(c_fit[idepth, :, :], origin="lower", vmin=-vmax, vmax=vmax, cmap="bwr")
    plt.axis("off")